# 第3章 メモリ

 - [1. OpenAI API キーの設定](#1-openai-api-キーの設定)
 - [2. 対話バッファメモリ](#2-対話バッファメモリ)
     - [2.1 英語版](#21-英語版)
         - [2.1.1 対話モデルの初期化](#211-対話モデルの初期化)
         - [2.1.2 第1回対話](#212-第1回対話)
         - [2.1.3 第2回対話](#213-第2回対話)
         - [2.1.4 第3回対話](#214-第3回対話)
         - [2.1.5 メモリキャッシュの確認](#215-メモリキャッシュの確認)
         - [2.1.6 メモリキャッシュへの直接コンテンツ追加](#216-メモリキャッシュへの直接コンテンツ追加)
     - [2.2 日本語版](#22-日本語版)
         - [2.1.1 対話モデルの初期化](#211-対話モデルの初期化)
         - [2.1.2 第1回対話](#212-第1回対話)
         - [2.1.3 第2回対話](#213-第2回対話)
         - [2.1.4 第3回対話](#214-第3回対話)
         - [2.1.5 メモリキャッシュの確認](#215-メモリキャッシュの確認)
         - [2.1.6 メモリキャッシュへの直接コンテンツ追加](#216-メモリキャッシュへの直接コンテンツ追加)
     - [2.3 まとめ](#23-まとめ)
 - [3. 対話バッファウィンドウメモリ](#3-対話バッファウィンドウメモリ)
     - [3.1 英語版](#31-英語版)
         - [3.1.1 ウィンドウメモリに2回の対話を追加](#311-ウィンドウメモリに2回の対話を追加)
         - [3.1.2 対話チェーンでのウィンドウメモリの応用](#312-対話チェーンでのウィンドウメモリの応用)
     - [3.2 日本語版](#32-日本語版)
         - [3.1.1 ウィンドウメモリに2回の対話を追加](#311-ウィンドウメモリに2回の対話を追加)
         - [3.1.2 対話チェーンでのウィンドウメモリの応用](#312-対話チェーンでのウィンドウメモリの応用)
 - [4. 対話トークンバッファメモリ](#4-対話トークンバッファメモリ)
     - [4.1 英語版](#41-英語版)
     - [4.2 日本語版](#42-日本語版)
     - [4.3 補足](#43-補足)
 - [5. 対話要約バッファメモリ](#5-対話要約バッファメモリ)
     - [5.1 英語版](#51-英語版)
         - [5.1.1 対話要約バッファメモリの使用](#511-対話要約バッファメモリの使用)
         - [5.1.2 対話要約バッファメモリに基づく対話チェーン](#512-対話要約バッファメモリに基づく対話チェーン)
     - [5.2 日本語版](#52-日本語版)
         - [5.2.1 対話要約バッファメモリの使用](#521-対話要約バッファメモリの使用)
         - [5.1.2 対話要約バッファメモリに基づく対話チェーン](#512-対話要約バッファメモリに基づく対話チェーン)

言語モデルと対話する際、それらは以前にあなたと行った対話の内容を覚えていません。これは、チャットボットなどのアプリケーションを構築する際に大きな問題となります - 十分にインテリジェントに見えないのです！したがって、このセクションでは、LangChainのメモリモジュール、つまり以前の対話を言語モデルに埋め込み、連続的な対話能力を持たせる方法について紹介します。

LangChainのメモリ（Memory）モジュールを使用すると、履歴チャットメッセージの保存と管理、特定のエンティティに関する知識の構築に役立ちます。これらのコンポーネントは複数回の対話にわたって情報を保存し、対話中に特定の情報とコンテキストを追跡することができます。

LangChainは複数のメモリタイプを提供しています。その中で、バッファメモリは最近のチャットメッセージを保持し、要約メモリは対話全体の要約を提供します。エンティティメモリは複数回の対話において特定のエンティティに関する情報を保持することができます。これらのメモリコンポーネントはすべてモジュラーで、他のコンポーネントと組み合わせて使用でき、ボットの対話管理能力を向上させます。メモリモジュールは簡単なAPI呼び出しでアクセスと更新が可能で、開発者が対話履歴の管理と維持をより簡単に実装できます。

この課程では主に4つのメモリモジュールを紹介しますが、その他のモジュールについてはドキュメントを参照して学習してください。
- 対話バッファメモリ（ConversationBufferMemory）
- 対話バッファウィンドウメモリ（ConversationBufferWindowMemory）
- 対話トークンバッファメモリ（ConversationTokenBufferMemory）
- 対話要約バッファメモリ（ConversationSummaryBufferMemory）

LangChainにおいて、メモリとは大規模言語モデル（LLM）の短期記憶を指します。なぜ短期記憶なのでしょうか？それは、LLMが訓練された後（一定の長期記憶を獲得した後）、そのパラメータはユーザーの入力によって変化することがないためです。ユーザーが訓練済みのLLMと対話する際、LLMは一時的にユーザーの入力と既に生成した出力を覚えて、以降の出力を予測しますが、モデルが出力を完了した後、以前のユーザーの入力と出力を「忘れて」しまいます。したがって、以前のこれらの情報はLLMの短期記憶としか呼べません。

LLMの短期記憶の保持時間を延長するには、外部メモリ方式を借りて記憶する必要があります。これにより、ユーザーとLLMの対話において、LLMができるだけユーザーとの履歴対話情報を知ることができるようになります。

## 1. OpenAI API キーの設定

[OpenAI アカウント](https://platform.openai.com/account/api-keys)にログインしてAPI キーを取得し、環境変数として設定してください。

- グローバル環境変数として設定したい場合は、[知乎文章](https://zhuanlan.zhihu.com/p/627665725)を参考にしてください。
- ローカル/プロジェクト環境変数として設定したい場合は、このファイルのディレクトリに`.env`ファイルを作成し、ファイルを開いて以下の内容を入力してください。

    <p style="font-family:verdana; font-size:12px;color:green">
    OPENAI_API_KEY="your_api_key" 
    </p>
  
  "your_api_key"をあなた自身のAPI キーに置き換えてください

In [1]:
import os

import openai
from dotenv import find_dotenv, load_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai.api_key = os.environ["OPENAI_API_KEY"]

## 2. 対話バッファメモリ

### 2.1 英語版

#### 2.1.1 対話モデルの初期化

In [2]:
from langchain.chains.conversation.base import ConversationChain
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [ ]:
# ここではtemperatureパラメータを0.0に設定し、生成される回答のランダム性を減らします。
# 毎回異なる新しいアイデアの回答を得たい場合は、このパラメータを増やしてみてください。
llm = ChatOpenAI(temperature=0.0) 

memory = ConversationBufferMemory()

# 新しいConversationChain Classインスタンスを作成
# verboseパラメータをTrueに設定すると、プログラムはより詳細な情報を出力し、より多くのデバッグまたは実行時情報を提供します。
# 反対に、verboseパラメータをFalseに設定すると、プログラムはより簡潔に実行され、重要な情報のみを出力します。
conversation = ConversationChain(llm=llm, memory = memory, verbose=True )

#### 2.1.2 第1回対話

予測（<font style="font-family:verdana; font-size:13.5px;color:#0055AA">predict</font>）を実行すると、いくつかのプロンプトが生成されます。以下で見られるように、「以下は人間とAIの間の友好的な対話で、AIは健談」などと言っています。これは実際にLangChainが生成したプロンプトで、システムが希望する友好的な対話を行い、対話を保存し、現在完了したモデルチェーンを促すためのものです。

In [4]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

#### 2.1.3 第2回対話

第2回対話を行うと、上記のプロンプトが保持されます

In [5]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1 + 1 equals 2. Is there anything else you would like to know?'

#### 2.1.4 第3回対話

以前の対話内容を記憶しているかを確認するため、前に言った内容（私の名前）を答えさせます。確かに正しい名前を出力しており、この対話チェーンは進行するにつれてどんどん長くなっていくことがわかります

In [6]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

#### 2.1.5 メモリキャッシュの確認

メモリキャッシュ（<font style="font-family:verdana; font-size:13.5px">memory.</font><font style="font-family:verdana; font-size:13.5px;color:#0055AA">buffer</font>）には現在までのすべての対話情報が保存されています

In [7]:
print(memory.buffer) 

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Andrew.


<font style="font-family:verdana; font-size:13.5px">memory.</font><font style="font-family:verdana; font-size:13.5px;color:#0055AA">load_memory_variables</font><font style="font-family:verdana; font-size:13.5px">({})</font>を使ってキャッシュ内の履歴メッセージを印刷することもできます。ここの`{}`は空の辞書で、より高度な機能がありユーザーがより複雑な入力を使用できますが、この短期コースでは説明しませんので、ここになぜ空の中括弧があるかは気にしないでください。

In [8]:
print(memory.load_memory_variables({}))

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1 + 1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Andrew."}


#### 2.1.6 メモリキャッシュへの直接コンテンツ追加

In [ ]:
memory = ConversationBufferMemory()  # 新しい空の対話バッファメモリを作成

In [ ]:
memory.save_context({"input": "Hi"}, {"output": "What's up"})  # バッファ領域に指定された対話の入出力を追加

In [ ]:
print(memory.buffer)  # バッファ領域の結果を確認

In [ ]:
print(memory.load_memory_variables({}))# メモリ変数を再度読み込み

新しいコンテンツを追加し続けると、対話履歴がすべて保存されます！

In [13]:
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [14]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

### 2.2 日本語版

#### 2.1.1 初始化对话模型

In [15]:
from langchain.chains.conversation.base import ConversationChain
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [16]:
llm = ChatOpenAI(temperature=0.0)  

memory = ConversationBufferMemory()

conversation = ConversationChain(llm=llm, memory = memory, verbose=True )

#### 2.1.2 第一轮对话

In [17]:
conversation.predict(input="你好, 我叫皮皮鲁")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好, 我叫皮皮鲁
AI:

> Finished chain.


' 你好皮皮鲁！我是一个人工智能助手，很高兴认识你。你有什么问题想问我吗？'

#### 2.1.3 第二轮对话

In [18]:
conversation.predict(input="1+1等于多少？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫皮皮鲁
AI:  你好皮皮鲁！我是一个人工智能助手，很高兴认识你。你有什么问题想问我吗？
Human: 1+1等于多少？
AI:

> Finished chain.


'1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？我可以帮助你解答。'

#### 2.1.4 第三轮对话

In [19]:
conversation.predict(input="我叫什么名字？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好, 我叫皮皮鲁
AI:  你好皮皮鲁！我是一个人工智能助手，很高兴认识你。你有什么问题想问我吗？
Human: 1+1等于多少？
AI: 1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？我可以帮助你解答。
Human: 我叫什么名字？
AI:

> Finished chain.


'抱歉，我不知道你的名字。你可以告诉我吗？我会记住的。'

#### 2.1.5 查看储存缓存

In [20]:
print(memory.buffer) 

Human: 你好, 我叫皮皮鲁
AI:  你好皮皮鲁！我是一个人工智能助手，很高兴认识你。你有什么问题想问我吗？
Human: 1+1等于多少？
AI: 1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？我可以帮助你解答。
Human: 我叫什么名字？
AI: 抱歉，我不知道你的名字。你可以告诉我吗？我会记住的。


In [21]:
print(memory.buffer)  

Human: 你好, 我叫皮皮鲁
AI:  你好皮皮鲁！我是一个人工智能助手，很高兴认识你。你有什么问题想问我吗？
Human: 1+1等于多少？
AI: 1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？我可以帮助你解答。
Human: 我叫什么名字？
AI: 抱歉，我不知道你的名字。你可以告诉我吗？我会记住的。


In [22]:
print(memory.load_memory_variables({}))

{'history': 'Human: 你好, 我叫皮皮鲁\nAI:  你好皮皮鲁！我是一个人工智能助手，很高兴认识你。你有什么问题想问我吗？\nHuman: 1+1等于多少？\nAI: 1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？我可以帮助你解答。\nHuman: 我叫什么名字？\nAI: 抱歉，我不知道你的名字。你可以告诉我吗？我会记住的。'}


In [23]:
memory.load_memory_variables({})

{'history': 'Human: 你好, 我叫皮皮鲁\nAI:  你好皮皮鲁！我是一个人工智能助手，很高兴认识你。你有什么问题想问我吗？\nHuman: 1+1等于多少？\nAI: 1加1等于2。这是一个基本的数学问题，答案是2。你还有其他问题吗？我可以帮助你解答。\nHuman: 我叫什么名字？\nAI: 抱歉，我不知道你的名字。你可以告诉我吗？我会记住的。'}

#### 2.1.6 直接添加内容到储存缓存

In [24]:
memory = ConversationBufferMemory()
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西'}

In [25]:
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [26]:
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西\nHuman: Not much, just hanging\nAI: Cool'}

In [27]:
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})

{'history': 'Human: 你好，我叫皮皮鲁\nAI: 你好啊，我叫鲁西西\nHuman: Not much, just hanging\nAI: Cool\nHuman: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

### 2.3 まとめ

大規模言語モデルでチャット対話を行う際、**大規模言語モデル自体は実際にはステートレスです。言語モデル自体は現在までの履歴対話を覚えていません**。API エンドポイントへの各呼び出しは独立しています。メモリ（Memory）は現在までのすべての用語や対話を保存し、それをLLMに入力または付加コンテキストとして提供して出力を生成できます。このように、次の対話を行う際に以前に何を言ったかを覚えているように見えます。

## 3. 対話バッファウィンドウメモリ
  
対話が長くなるにつれて、必要なメモリ量も非常に大きくなります。大量のトークンをLLMに送信するコストも高くなり、これがAPI呼び出し料金が通常処理する必要があるトークン数に基づいて課金される理由です。
  
上記の問題に対して、LangChainは履歴対話を保存する便利なメモリ方式もいくつか提供しています。その中で、対話バッファウィンドウメモリはウィンドウサイズの対話のみを保持します。最近のn回のインタラクションのみを使用します。これは最近のインタラクションのスライディングウィンドウを保持するために使用でき、バッファが大きくなりすぎないようにできます

### 3.1 英文版

#### 3.1.1 添加两轮对话到窗口储存

In [28]:
from langchain.memory import ConversationBufferWindowMemory

# k 为窗口参数，k=1表明只保留一个对话记忆
memory = ConversationBufferWindowMemory(k=1)  

In [29]:
# 向memory添加两轮对话
memory.save_context({"input": "Hi"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

In [30]:
# 并查看记忆变量当前的记录
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

#### 3.1.2 在对话链中应用窗口储存

In [31]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=llm, memory=memory, verbose=False  )

注意此处！由于这里用的是一个窗口的记忆，因此只能保存一轮的历史消息，因此AI并不能知道你第一轮对话中提到的名字，他最多只能记住上一轮（第二轮）的对话信息

In [32]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [33]:
conversation.predict(input="What is 1+1?")

'1+1 equals 2. Is there anything else you would like to know?'

In [34]:
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to ask?"

### 3.2 中文版

#### 3.1.1 添加两轮对话到窗口储存

In [35]:
from langchain.memory import ConversationBufferWindowMemory

# k=1表明只保留一个对话记忆
memory = ConversationBufferWindowMemory(k=1)  
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})

{'history': 'Human: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}

#### 3.1.2 在对话链中应用窗口储存

In [36]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=llm, memory=memory, verbose=False  )
print(conversation.predict(input="你好, 我叫皮皮鲁"))
print(conversation.predict(input="1+1等于多少？"))
print(conversation.predict(input="我叫什么名字？"))

 你好，皮皮鲁！很高兴认识你。我是一个人工智能助手，可以回答你的问题或者和你聊天。有什么我可以帮助你的吗？
1加1等于2。这是一个基本的数学问题，答案是2。如果你有任何其他数学问题，都可以问我哦！
抱歉，我不知道你的名字。我只是一个AI程序，无法获取你的个人信息。如果你有其他问题，我会尽力回答的哦！


## 四、对话token缓存储存

使用对话token缓存记忆，内存将限制保存的token数量。如果token数量超出指定数目，它会切掉这个对话的早期部分
以保留与最近的交流相对应的token数量，但不超过token限制。

In [37]:
from langchain.memory import ConversationTokenBufferMemory

### 4.1 英文版
添加对话到Token缓存储存,限制token数量，进行测试

In [38]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

In [39]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

可以看到前面超出的的token已经被舍弃了！！！

### 4.2 中文版

In [40]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "朝辞白帝彩云间，"}, {"output": "千里江陵一日还。"})
memory.save_context({"input": "两岸猿声啼不住，"}, {"output": "轻舟已过万重山。"})
memory.load_memory_variables({})

{'history': 'AI: 轻舟已过万重山。'}

### 4.3 补充

ChatGPT使用一种基于字节对编码（Byte Pair Encoding，BPE）的方法来进行tokenization（将输入文本拆分为token）。BPE是一种常见的tokenization技术，它将输入文本分割成较小的子词单元。 

OpenAI在其官方GitHub上公开了一个最新的开源Python库 [tiktoken](https://github.com/openai/tiktoken)，这个库主要是用来计算tokens数量的。相比较HuggingFace的tokenizer，其速度提升了好几倍。

具体token计算方式,特别是汉字和英文单词的token区别，具体课参考[知乎文章](https://www.zhihu.com/question/594159910) 。

## 五、对话摘要缓存储存

对话摘要缓存储存，**使用LLM编写到目前为止历史对话的摘要**，并将其保存

In [41]:
from langchain.memory import ConversationSummaryBufferMemory

### 5.1 英文版

#### 5.1.1 使用对话摘要缓存储存

创建一个长字符串，其中包含某人的日程安排

In [42]:
# 创建一个长字符串
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

# 使用对话摘要缓存记忆
llm = ChatOpenAI(temperature=0.0)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100) 
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context(
    {"input": "What is on the schedule today?"}, {"output": f"{schedule}"}
)

In [43]:
print(memory.load_memory_variables({})['history'])

System: The human and AI exchange greetings and discuss the day's schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the power of LangChain as a tool.


#### 5.1.2 基于对话摘要缓存储存的对话链
基于上面的对话摘要缓存储存，新建一个对话链

In [44]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [45]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the day's schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the power of LangChain as a tool.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show would be a live demonstration of how LangChain can accurately translate complex technical documents from one language to another in real-time. This would showcase the speed, accuracy, and efficiency of the language processing capabilities of LangChain. Additionally, demonstrating how LangChain can seamlessly integrate with other AI tools and platforms would highlight its versatility and compatibility with existing systems.'

In [46]:
print(memory.load_memory_variables({})['history'])

System: The human and AI exchange greetings and discuss the day's schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the power of LangChain as a tool.
Human: What would be a good demo to show?
AI: A good demo to show would be a live demonstration of how LangChain can accurately translate complex technical documents from one language to another in real-time. This would showcase the speed, accuracy, and efficiency of the language processing capabilities of LangChain. Additionally, demonstrating how LangChain can seamlessly integrate with other AI tools and platforms would highlight its versatility and compatibility with existing systems.


### 5.2 中文版

#### 5.2.1 使用对话摘要缓存储存

创建一个长字符串，其中包含某人的日程安排

In [47]:
# 创建一个长字符串
schedule = "在八点你和你的产品团队有一个会议。 \
你需要做一个PPT。 \
上午9点到12点你需要忙于LangChain。\
Langchain是一个有用的工具，因此你的项目进展的非常快。\
中午，在意大利餐厅与一位开车来的顾客共进午餐 \
走了一个多小时的路程与你见面，只为了解最新的 AI。 \
确保你带了笔记本电脑可以展示最新的 LLM 样例."

llm = ChatOpenAI(temperature=0.0)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "你好，我叫皮皮鲁"}, {"output": "你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.save_context({"input": "今天的日程安排是什么？"}, {"output": f"{schedule}"})

In [48]:
print(memory.load_memory_variables({})['history'])

System: The human and AI introduce themselves in Chinese and become friends. They plan their schedule for the day, including a meeting with the product team, working on LangChain, and having lunch with a customer interested in AI. The AI emphasizes the importance of being prepared to showcase the latest LLM examples.


#### 5.1.2 基于对话摘要缓存储存的对话链
基于上面的对话摘要缓存储存，新建一个对话链

In [49]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

In [50]:
conversation.predict(input="展示什么样的样例最好呢？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI introduce themselves in Chinese and become friends. They plan their schedule for the day, including a meeting with the product team, working on LangChain, and having lunch with a customer interested in AI. The AI emphasizes the importance of being prepared to showcase the latest LLM examples.
Human: 展示什么样的样例最好呢？
AI:

> Finished chain.


'最好展示一些具有创新性和实用性的语言模型样例，比如可以展示LangChain在不同语言之间的翻译效果，或者展示其在文本生成和对话系统方面的应用。这样可以让客户更直观地了解LangChain的潜力和优势。'

In [51]:
memory.load_memory_variables({})  # 摘要记录更新了

{'history': 'System: The human and AI introduce themselves in Chinese and become friends. They plan their schedule for the day, including a meeting with the product team, working on LangChain, and having lunch with a customer interested in AI. The AI emphasizes the importance of being prepared to showcase the latest LLM examples. The human asks what kind of examples would be best to showcase.\nAI: 最好展示一些具有创新性和实用性的语言模型样例，比如可以展示LangChain在不同语言之间的翻译效果，或者展示其在文本生成和对话系统方面的应用。这样可以让客户更直观地了解LangChain的潜力和优势。'}